# Importowanie bibliotek

W projekcie wykorzystano następujące biblioteki:
 - TensorFlow i Keras: Główne narzędzia do budowy, trenowania i zarządzania modelami głębokiego uczenia.
 - Keras Tuner (keras_tuner): Narzędzie do optymalizacji hiperparametrów. Pozwala na przeszukiwanie przestrzeni parametrów, takich jak liczba neuronów, dropout czy tempo uczenia.
 - Scikit-learn: Zestaw narzędzi do przetwarzania danych i analizy
 - Pandas: Umożliwia manipulację i analizę danych w formacie tabelarycznym. Wykorzystywany do ładowania, filtrowania i wstępnego przetwarzania danych.
 - NumPy: Biblioteka numeryczna używana do obsługi macierzy i wektorów, zapewniająca szybką manipulację danymi.
 - TensorFlow (tf): Umożliwia zaawansowane obliczenia tensorowe oraz wykorzystanie GPU do przyspieszenia procesu uczenia modeli.

In [6]:
import keras_tuner as kt
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Wczytanie i przygotowanie danych
Źródło danych:
 - Dane zostały wczytane z pliku CSV (data.csv) za pomocą funkcji pd.read_csv.

Filtracja danych:
- Wybrano tylko numeryczne kolumny, które są istotne dla procesu klasyfikacji:
     - Size(L) – rozmiar partii piwa.
     - OG i FG – gęstość początkowa i końcowa piwa.
     - ABV – zawartość alkoholu w procentach.
     - IBU – jednostki goryczki.
     - Color – barwa piwa.
     - BoilSize – objętość cieczy do gotowania.
     - BoilTime – czas gotowania.
     - BoilGravity – gęstość brzeczki przed gotowaniem.
     - Efficiency – wydajność procesu warzenia.

Usuwanie braków danych:
 - Wszystkie brakujące wartości w danych numerycznych zostały uzupełnione średnią z danej kolumny (fillna(X.mean())).

Kodowanie etykiet:
- Style piwa (kolumna Style) zostały przekodowane na wartości całkowite za pomocą LabelEncoder, aby były odpowiednie do analizy i modelowania.

Skalowanie danych:
 - Dane numeryczne zostały znormalizowane za pomocą StandardScaler, co zapewnia, że wszystkie cechy mają średnią równą 0 i odchylenie standardowe równe 1. Taki proces poprawia wydajność algorytmów uczenia maszynowego.

Podział danych:
- Zestaw danych został podzielony na zbiór treningowy i testowy w proporcji 80% do 20% za pomocą train_test_split.

Przygotowanie etykiet do klasyfikacji wieloklasowej:
- Etykiety zostały przekonwertowane do formatu kategorycznego (one-hot encoding) za pomocą tf.keras.utils.to_categorical, gdzie liczba klas odpowiada liczbie unikalnych stylów piwa w danych.

In [7]:
data = pd.read_csv('../data.csv')

numeric_columns = ['Size(L)', 'OG', 'FG', 'ABV', 'IBU', 'Color', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = data[numeric_columns].copy()

X = X.fillna(X.mean())

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Style'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

num_classes = len(np.unique(y))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("Kształt danych treningowych:", X_train.shape)
print("Kształt danych testowych:", X_test.shape)
print("Kształt etykiet treningowych:", y_train.shape)
print("Kształt etykiet testowych:", y_test.shape)

Kształt danych treningowych: (59088, 10)
Kształt danych testowych: (14773, 10)
Kształt etykiet treningowych: (59088, 176)
Kształt etykiet testowych: (14773, 176)


# Struktura modelu
Zbudowano model sekwencyjny z następującymi warstwami:
- Warstwa wejściowa:
  - Liczba neuronów dynamicznie dobierana (32–512, krok 32).
  - Aktywacja ReLU.
- Batch Normalization: Stabilizacja procesu uczenia przez normalizację wsadową.
- Dropout: Regularizacja z dynamicznie dobieraną wartością (0–50%, krok 10%).
- Warstwy ukryte:
  - Pierwsza warstwa: Liczba neuronów dynamicznie dobierana (32–512, krok 32), aktywacja ReLU.
  - Druga warstwa: Jak wyżej.
  - Obie warstwy ukryte wyposażono w Batch Normalization i Dropout.
- Warstwa wyjściowa: Liczba neuronów równa liczbie klas (style piwa), aktywacja softmax.

Kompilacja modelu
- Optymalizator: Adam z dynamicznie dobieraną szybkością uczenia (0.0001–0.01, skala logarytmiczna).
- Funkcja straty: categorical_crossentropy.
- Metryka: accuracy (dokładność).

In [8]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_scaled.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Proces tuningu hiperparametrów
Metoda tuningu: Bayesian Optimization (optymalizacja bayesowska).
- Hiperparametry:
  - Liczba neuronów w warstwach: 32–512 (krok 32).
  - Dropout: 0–50% (krok 10%).
  - Learning rate: 0.0001–0.01 (skala logarytmiczna).
- Liczba prób: 20 (maksymalna liczba konfiguracji testowych).

Proces uczenia
- Parametry treningu:
  - Maksymalna liczba epok: 20.
  - Batch size: 32.
- Callbacki:
  - EarlyStopping: Monitorowanie straty walidacji, zatrzymanie po 5 epokach bez poprawy.
  - TensorBoard: Monitorowanie wyników uczenia w czasie rzeczywistym.

In [9]:
tuner = kt.BayesianOptimization(build_model,
                                 objective='val_accuracy',
                                 max_trials=20, 
                                 directory='logs/fit', 
                                 project_name='klasyfikacja_styli_piwa')

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

log_dir = "logs/fit/klasyfikacja_styli_piwa"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

tuner.search(X_train,
             y_train, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback], 
             batch_size=32)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 20 Complete [00h 00m 54s]
val_accuracy: 0.34117448329925537

Best val_accuracy So Far: 0.3415129482746124
Total elapsed time: 00h 16m 48s


In [10]:
best_model = build_model(best_hps)

best_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 512)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 176)            │        90,288 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,672 (1.38 MB)

 Trainable params: 360,624 (1.38 MB)

 Non-trainable params: 2,048 (8.00 KB)

# Wyniki

- Najlepszy model: Osiągnął dokładność walidacyjną 34.15% w ostatniej epoce.
- Finalna dokładność walidacyjna: 34%.
- Strata walidacyjna: Systematycznie spadała, stabilizując się w zakresie 2.7–2.8 po kilku epokach.

In [ ]:
history = best_model.fit(X_train,
                         y_train, 
                         epochs=20, 
                         validation_split=0.2, 
                         callbacks=[early_stopping, tensorboard_callback],
                         batch_size=32)

final_val_accuracy = history.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {final_val_accuracy:.2f}")

# Zapisywanie Modelu

In [12]:
best_model.save('../models/tuned_model_klasyfikacji_piwa.h5')

loaded_model = keras.models.load_model('../models/tuned_model_klasyfikacji_piwa.h5')

loaded_model.summary()

loaded_test_loss, loaded_test_accuracy = loaded_model.evaluate(X_train, y_train)

print(f'Loaded Test Loss: {loaded_test_loss}')
print(f'Loaded Test Accuracy: {loaded_test_accuracy}')

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 512)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 176)            │        90,288 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,674 (1.38 MB)

 Trainable params: 360,624 (1.38 MB)

 Non-trainable params: 2,048 (8.00 KB)

 Optimizer params: 2 (12.00 B)

1847/1847 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3521 - loss: 2.5325
Loaded Test Loss: 2.5613596439361572
Loaded Test Accuracy: 0.3496987521648407
